In [1]:
from IPython.display import display, SVG
from typing import Optional, Sequence
import dataclasses
import math

from typesetting import ParagraphItem, ParagraphItemType, Font, Glyph, hyphenate, text_to_paragraph_items, MAX_PENALTY, paragraph_item_running_sums, greedy_line_breaks
from typesetting.cairo import fill_glyphs_at

Source:

http://www.eprg.org/G53DOC/pdfs/knuth-plass-breaking.pdf

In [2]:
from faker import Faker
import textwrap

Faker.seed(0xdeadbeef)
lorem = hyphenate("".join([
    "Difficult Quick Foxes with 1034 boxes. ",
    Faker(["la"]).paragraph(50)
]))
print(f"Generated lorem ipsum:\n\n{textwrap.fill(textwrap.shorten(lorem, 200), 72)}")
print(f"\nRepr of first part:\n{textwrap.shorten(lorem, 72)!r}")

Generated lorem ipsum:

Dif­fi­cult Quick Foxes with 1034 boxes. Iusto no­bis in­cidunt nos­trum
quas oc­cae­cati. Ex fa­cilis pariatur quam rep­re­hen­derit fugit
magni. Eius do­lo­rum do­lo­rum maxime hic. Il­lum [...]

Repr of first part:
'Dif\xadfi\xadcult Quick Foxes with 1034 boxes. Iusto no\xadbis in\xadcidunt [...]'


In [29]:
rm_face_path = "EBGaramond-VariableFont_wght.ttf"
rm_size = 14
rm_font = Font(path=rm_face_path, em_size=[rm_size, rm_size], features=["onum"])

sm_rm_size = rm_size * 0.8
sm_rm_font = Font(path=rm_face_path, em_size=[sm_rm_size, sm_rm_size], features=["onum"])

In [30]:
import uniseg.linebreak
print(textwrap.fill("|".join(uniseg.linebreak.line_break_units(lorem)), 80))

Dif­|fi­|cult |Quick |Foxes |with |1034 |boxes. |Iusto |no­|bis |in­|cidunt
|nos­|trum |quas |oc­|cae­|cati. |Ex |fa­|cilis |pariatur |quam
|rep­|re­|hen­|derit |fugit |magni. |Eius |do­|lo­|rum |do­|lo­|rum |maxime
|hic. |Il­|lum |nemo |neque |con­|secte­|tur |volup­|ta­|tum |per­|spi­|ci­|atis
|ab. |Op­|tio |mo­|les­|tias |no­|bis |er­|ror |facere |libero. |Harum |ip­|sam
|na­|tus |rep­|re­|hen­|derit |do­|lore |as­|sumenda |unde. |Tene­|tur |ab
|cor­|rupti |rem |modi |quia |fuga. |Ali­|quam |pariatur |do­|lores |esse
|con­|secte­|tur |ipsa |fugit. |Beatae |per­|fer­|endis |alias |deb­|itis. |Eius
|mi­|nus |volup­|tas |amet |quis |quae. |Fuga |cor­|rupti |lau­|dan­|tium
|in­|cidunt |in |ipsa |tem­|pora. |Unde |ver­|i­|tatis |quia |iure |volup­|tatem
|fugit |culpa. |Min­|ima |ex­|cep­|turi |saepe |vero |sit |of­|fi­|cia. |At |eum
|rep­|re­|hen­|derit |ver­|i­|tatis |quos. |Neque |pos­|simus |mo­|les­|tiae
|ver­|i­|tatis |mo­|les­|tias |autem |fu­|giat |min­|ima. |Lau­|dan­|tium
|sapi­

In [31]:
para_items = list(text_to_paragraph_items(lorem, rm_font))

if False:
    print("First few items:")
    print("\n".join(textwrap.indent(f"{item!r}", "  ") for item in para_items[:10]))
    print("Final few items:")
    print("\n".join(textwrap.indent(f"{item!r}", "  ") for item in para_items[-10:]))

In [32]:
import io
from IPython.display import display, SVG
import cairo
from typing import Iterable

def render_lines(ctx: cairo.Context, line_break_idxs: Iterable[int], x0: float, y0: float):
    sums = list(paragraph_item_running_sums(para_items))
    line_start_idx = 0
    for line_idx, line_end_idx in enumerate(line_break_idxs):
        y = line_height*line_idx + y0
        line_items = []
        x = x0
        running_text, running_width = [], 0.0
    
        # What delta needs to be taken up by glue
        natural_width = sums[line_end_idx].width - sums[line_start_idx].width
        if para_items[line_end_idx].item_type == ParagraphItemType.PENALTY:
            natural_width += para_items[line_end_idx].width
        total_stretch = sums[line_end_idx].stretchability - sums[line_start_idx].stretchability
        total_shrink = sums[line_end_idx].shrinkability - sums[line_start_idx].shrinkability
        delta = line_width - natural_width
    
        # Total slack for delta
        if delta > 0.0:
            adjustment_ratio = delta / total_stretch
        else:
            adjustment_ratio = delta / total_shrink

        fill_glyphs_at(ctx, x + line_width + rm_font.em_size[0], y, sm_rm_font.shape(f"{adjustment_ratio:.2f}"), sm_rm_font)
        
        for item_idx in range(line_start_idx, line_end_idx+1):
            item = para_items[item_idx]
            if item_idx != line_end_idx and item.item_type == ParagraphItemType.PENALTY:
                continue
            if item_idx == line_end_idx and item.item_type == ParagraphItemType.GLUE:
                continue
            running_width += item.width
            if item.item_type == ParagraphItemType.GLUE:
                text = "".join(running_text)
                fill_glyphs_at(ctx, x, y, rm_font.shape(text), rm_font)
                x += running_width
                if adjustment_ratio > 0:
                    x += item.stretchability * adjustment_ratio
                else:
                    x += item.shrinkability * adjustment_ratio
    
                running_text, running_width = [], 0.0
            else:
                running_text.append(item.text)
        text = "".join(running_text)
        if text != "":
            fill_glyphs_at(ctx, x, y, rm_font.shape(text), rm_font)
        line_start_idx = line_end_idx+1

In [33]:
line_width = 5 * 72
greedy_line_break_idxs = list(greedy_line_breaks(para_items, line_width))
line_height = rm_size * 1.1
paper_size = (line_width + rm_font.em_size[0] * 5, line_height * (len(greedy_line_break_idxs) + 2))

In [34]:
def draw_page(ctx: cairo.Context, line_break_idxs: Sequence[int], line_width: float, line_height: float):
    ctx.set_source_rgb(1, 1, 1)
    ctx.rectangle(0, 0, paper_size[0], paper_size[1])
    ctx.fill()

    ctx.set_source_rgba(1, 0, 0, 0.5)
    ctx.set_line_width(1.0)

    ctx.move_to(rm_size, 0)
    ctx.rel_line_to(0, paper_size[1])
    ctx.move_to(rm_size + line_width, 0)
    ctx.rel_line_to(0, paper_size[1])
    ctx.close_path()
    ctx.stroke()

    ctx.set_source_rgba(0, 0, 1, 0.25)
    for line_idx in range(len(line_break_idxs)+1):
        ctx.move_to(rm_size, (line_idx+1) * line_height)
        ctx.rel_line_to(line_width, 0)
    ctx.stroke()
    
    ctx.set_source_rgb(0, 0, 0)
    render_lines(ctx, line_break_idxs, rm_size, 2*line_height)

svg_file = io.BytesIO()
with cairo.SVGSurface(svg_file, paper_size[0], paper_size[1]) as surface:
    ctx = cairo.Context(surface)
    draw_page(ctx, greedy_line_break_idxs, line_width, line_height)
display(SVG(svg_file.getvalue()))

with cairo.PDFSurface("example.pdf", paper_size[0], paper_size[1]) as surface:
    ctx = cairo.Context(surface)
    draw_page(ctx, greedy_line_break_idxs, line_width, line_height)